In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [13]:
# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 6000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 128  # embedding size
n_head = 8  # attention heads
n_layer = 8  # transformer layers
dropout = 0.0

In [3]:
torch.manual_seed(1337)

In [4]:
#read it in to inspect it
with open('/content/drive/MyDrive/dataSet_for_practice/SMSSpamCollection.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [5]:
print('Length of the dataset in characters : ', len(text))

Length of the dataset in characters :  477203


Tokenization

In [6]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text))) # Unique characters in text
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) } # Convert string to list of integers
itos = { i:ch for i,ch in enumerate(chars) } # Convert list of integers to string

encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

In [7]:
# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long) #covert text into tensor
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

Data Batching

In [8]:
# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [9]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x



In [10]:
# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # # Combine token and positional embeddings (B,T,C)
        x = self.blocks(x) # Pass through Transformer blocks(B,T,C)
        x = self.ln_f(x) # Layer normalization(B,T,C)
        logits = self.lm_head(x) # Get predictions for next token (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            #Reshape logits and targets
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)

In [14]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)


for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()




step 0: train loss 1.8243, val loss 1.8793
step 100: train loss 1.8166, val loss 1.8706
step 200: train loss 1.8210, val loss 1.8569
step 300: train loss 1.8248, val loss 1.8713
step 400: train loss 1.8009, val loss 1.8542
step 500: train loss 1.8228, val loss 1.8761
step 600: train loss 1.8134, val loss 1.8657
step 700: train loss 1.8001, val loss 1.8552
step 800: train loss 1.7906, val loss 1.8374
step 900: train loss 1.7880, val loss 1.8571
step 1000: train loss 1.7999, val loss 1.8370
step 1100: train loss 1.7739, val loss 1.8507
step 1200: train loss 1.7582, val loss 1.8400
step 1300: train loss 1.7729, val loss 1.8279
step 1400: train loss 1.7643, val loss 1.8404
step 1500: train loss 1.7793, val loss 1.8288
step 1600: train loss 1.7617, val loss 1.8274
step 1700: train loss 1.7662, val loss 1.8142
step 1800: train loss 1.7580, val loss 1.8343
step 1900: train loss 1.7611, val loss 1.8335
step 2000: train loss 1.7549, val loss 1.8181
step 2100: train loss 1.7432, val loss 1.8155


In [15]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=1100)[0].tolist()))

	I i've saoke you's so i at workin blather GOOD
ham	Where undirteed bedno.mis we serring them teacing the does go not gonnah.........  maya i come comy!
ham	Feee I 3 wors.co.uk/don 0905809996687728.36 16.504 Txt to 6146536.Msg9287Q
ham	1d; ...!!
ham	K..k.i lovaing to was that you get seath by poly to showing ...
ham	I already getting send 150p
ham	Lol)hone round days wot go u £100 hole :(at phones that u don't be our 2f I himages but I quotol coloved ently netcup and ? comuch at it please  &lt;#&gt;,gd
ham	No someone-The wrare. With my job days coming fine n satemo oh..
ham	I hope the money I'll bake ur lots is west?
ham	id we go does roup..i've says in ancount i'm getting the cauuse with off:)when i get that I wantled? To life prazu
ham	Enjoy callin be  2 shower to acount only later
ham	I cops real. When me you hello in with meet they cext?
ham	Shands is pick. GONK, My 1WEN Now  &lt;#&gt;   elseouss
ham	Nays a gring to time : 
ham	I was we was ..... Jus love! When't i you finis and te